In [1]:
import threading
import time
import os

In [3]:
def decodelzw(compressed_data, rank):
    global NEXT_CODE
    decompressed_data = []
    string = ''
    print('|' * 10, f'HILO {rank} DECODIFICANDO -> {compressed_data}')
    for code in compressed_data:
        if code not in DICTIONARY:
            DICTIONARY[code] = string + (string[0])
        decompressed_data += DICTIONARY[code]
        if not(len(string) == 0):
            DICTIONARY[NEXT_CODE] = string + (DICTIONARY[code][0])
            NEXT_CODE += 1
        string = DICTIONARY[code]
    print('=' * 10, f'RESULTADO HILO {rank}  -> {decompressed_data}')
    DECODE_RESULT[rank] = decompressed_data

In [4]:
def parse_compressed_data(compressed_data):
    compressed_data = [i.split(',') for i in compressed_data.split('\n')]
    for i in range(len(compressed_data)):
        for j in range(len(compressed_data[i])):
            compressed_data[i][j] = int(compressed_data[i][j])
    return compressed_data

In [5]:
def open_compressed_data(file):
    file = open(file, 'r')
    compressed_data = file.read()
    parsed_compressed_data = parse_compressed_data(compressed_data)
    return parsed_compressed_data

In [6]:
# Paralelizar decodificacion
start_time = time.time()
BITS = 17

print('~' * 10, 'ABRIENDO ARCHIVO DE CODIFICACION')
RESULT = open_compressed_data('encoded_data.txt')
print('~' * 10, 'INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII')
DICTIONARY_SIZE = pow(2,int(BITS-1))
DICTIONARY = dict([(x, chr(x)) for x in range(DICTIONARY_SIZE)])

MAXIMUM_SIZE = pow(2,int(BITS)) 
NEXT_CODE = 256

P = 4
DECODE_RESULT = [[] for p in range(P)]
print('~' * 10, f'UTILIZANDO {P} HILOS')

for rank in range(P):
    # Ejecutar en paralelo
    args = (RESULT[rank], rank)
    t = threading.Thread(target=decodelzw, args=args)
    t.start()
    t.join()

print('~' * 10, 'UNIFICANDO RESULTADOS DECODIFICADOS')
DECODE_RESULT = ''.join(sum(DECODE_RESULT, []))
print('~' * 10, f'RESULTADO FINAL DE DECODIFICACION => {DECODE_RESULT}')
print('~' * 10, "TIEMPO DE EJECUCION:  %s " % (round((time.time() - start_time),2)*1000), " Mili-segundos")

print('~' * 10, 'GUARDANDO DE CODIFICACION')
output_file = open('decoded_data.txt', 'w+')
output_file.write(DECODE_RESULT)
output_file.close()

decodedFile = os.path.getsize('decoded_data.txt')
print('~' * 10, 'TAMAÑO DEL ARCHIVO CODIFICADO: ', decodedFile, "BYTES")

#GUARDAR TIEMPO TXT
size = os.path.getsize('data.txt')
output = open('Tiempos/tiempo_pd-%s.txt' % (size), 'w+')
output_string = str(round((time.time() - start_time),2)*1000)
output.write(output_string)
output.close()

~~~~~~~~~~ ABRIENDO ARCHIVO DE CODIFICACION
~~~~~~~~~~ INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII
~~~~~~~~~~ UTILIZANDO 4 HILOS
|||||||||| HILO 0 DECODIFICANDO -> [116, 104, 65536, 101, 103, 97, 109, 101]
========== RESULTADO HILO 0  -> ['t', 'h', 'h', 'h', 'e', 'g', 'a', 'm', 'e']
|||||||||| HILO 1 DECODIFICANDO -> [105, 110, 65538, 109, 105, 100, 100]
========== RESULTADO HILO 1  -> ['i', 'n', 'n', 'n', 'm', 'i', 'd', 'd']
|||||||||| HILO 2 DECODIFICANDO -> [108, 101, 111, 102, 65538, 110, 105]
========== RESULTADO HILO 2  -> ['l', 'e', 'o', 'f', 'n', 'n', 'n', 'i']
|||||||||| HILO 3 DECODIFICANDO -> [103, 65537, 109, 97, 100, 97, 102, 97, 99, 107, 97]
========== RESULTADO HILO 3  -> ['g', 'g', 'g', 'm', 'a', 'd', 'a', 'f', 'a', 'c', 'k', 'a']
~~~~~~~~~~ UNIFICANDO RESULTADOS DECODIFICADOS
~~~~~~~~~~ RESULTADO FINAL DE DECODIFICACION => thhhegameinnnmiddleofnnnigggmadafacka
~~~~~~~~~~ TIEMPO DE EJECUCION:  1140.0   Mili-segundos
~~~~~~~~~~ GUARDANDO DE CODIFICACION
~~~~~~~~~~